In [13]:
import numpy as np

In [14]:
import matplotlib.pyplot as plt

## Codigo ensamblador que efectua el producto interno entre dos vectores float

In [15]:
%%file asmFloatInnerProd.asm

global asmFloatInnerProd
    section .text

asmFloatInnerProd:
    xorpd xmm0, xmm0
    xorpd xmm1, xmm1
    xorpd xmm2, xmm2
    cmp rdx, 0
    je done
next:
    movss xmm0, [rdi]
    movss xmm1, [rsi]
    mulss xmm0, xmm1
    addss xmm2, xmm0
    add rdi, 4
    add rsi, 4
    sub rdx, 1
    jnz next 
done:
    movss xmm0, xmm2
    ret


Writing asmFloatInnerProd.asm


## Archivo de cabecera de la función en ensamblador para explicitar los parametros y sus tipos de dato

In [16]:
%%file asmFloatInnerProd.c
extern float asmFloatInnerProd(float *v1, float *v2, int N);

Writing asmFloatInnerProd.c


## Reglas para generar la libreria compartida que se vinculara con python

In [17]:
! nasm -f elf64 asmFloatInnerProd.asm -o asmFloatInnerProd.o
! gcc -shared asmFloatInnerProd.o asmFloatInnerProd.c -o asmFloatInnerProd.so

In [18]:
import ctypes

## Funcion que vincula la libreria compartida en ASM con Python

In [19]:
def ctypes_asmFloatInnerProd():
    # indicar la ruta de la shared library
    libfile = './asmFloatInnerProd.so'

    # cargar la shared library
    lib = ctypes.CDLL(libfile)

    # tipo de dato de los argumentos
    lib.asmFloatInnerProd.argtypes = [
        np.ctypeslib.ndpointer(dtype=np.float32),
        np.ctypeslib.ndpointer(dtype=np.float32),
        ctypes.c_int
    ]
    
    #Tipo de dato que devuelve
    lib.asmFloatInnerProd.restype = ctypes.c_float
    
    # funcion configurada
    return lib.asmFloatInnerProd

In [20]:
%%file cFloatInnerProd.c

float cFloatInnerProd(float *v1, float *v2, int N){
    float sum = 0;
    for (int i = 0; i < N; i++){
        sum += v1[i] * v2[i];
    }
    return = sum;
}

Writing cFloatInnerProd.c


## Reglas para generar la libreria compartida de código en C

In [21]:
# para generar el object file
! gcc -c -Wall -Werror -fpic cFloatInnerProd.c
#muestre errorres, resalte errores, pic (position independend code)

# para crear la shared library
! gcc -shared cFloatInnerProd.o -o cFloatInnerProd.so

cFloatInnerProd.c: In function ‘cFloatInnerProd’:
cFloatInnerProd.c:9:12: error: expected expression before ‘=’ token
    9 |     return = sum;
      |            ^
cFloatInnerProd.c:10:1: error: control reaches end of non-void function [-Werror=return-type]
   10 | }
      | ^
cc1: all warnings being treated as errors
gcc: error: cFloatInnerProd.o: No such file or directory
gcc: fatal error: no input files
compilation terminated.


## Funcion que vincula la libreria compartida en C con Python

In [10]:
def ctypes_cFloatInnerProd():
    # indicar la ruta de la shared library
    libfile = './cFloatInnerProd.so'

    # cargar la shared library
    lib = ctypes.CDLL(libfile)

    # tipo de dato de los argumentos
    lib.asmFloatInnerProd.argtypes = [
        np.ctypeslib.ndpointer(dtype=np.float32),
        np.ctypeslib.ndpointer(dtype=np.float32),
        ctypes.c_int
    ]
    
    #Tipo de dato que devuelve
    lib.cFloatInnerProd.restype = ctypes.c_float
    
    # funcion configurada
    return lib.cFloatInnerProd

## Prueba de las dos funciones

In [11]:
# tam
size = 2048

# entradas
x = np.random.rand(size,1).astype(np.float32)
y = np.random.rand(size,1).astype(np.float32)

## Limpieza de archivos

In [12]:
! rm *.o
! rm *.so
! rm *.asm
! rm *.c